# INF201 Lecture No 9

### 3 November 2025

## Today's topics

1. Object-oriented programming
    1. Review: Key ideas
    1. Defining new data types
        1. Review: Defining mathematical operations
        1. Defining comparisons
    1. Names, namespaces and scopes
    1. Copying objects
1. Staying in control
    1. Assertions
    1. Exceptions
    1. Testing

# Object-oriented programming

----------

## Review

### Key ideas
- Idea 1: Combine data and operations into new data types
    - A (user-defined) data type is a *class*
    - Each *object* is an *instance* of a class
    - Classes and objects have *attributes*
        - *Methods* are functions operating on objects
        - *Fields* are data (variables) contained in objects
        - Methods and fields are also known as *member functions* and *member variables*  
- Idea 2: Allow modification and extension of data types
    - *Inheritance*: Define a *subclass* based on a *superclass* and adapt or extend it
- Idea 3: Expose an interface, hide the implementation
    - *Interface*: Methods "advertised" for public use
    - *Implementation*: Does the actual work
    - Also know as *encapsulation*
    

-------------------

## Defining new data types

- OO Idea 1: Combine data and behavior into *new data types*
- Problem: How to make our classes behave more like built-in data types
    - nice printing
    - comparison between instances (e.g., sorting `Member`s)
    - mathematical operations (e.g., computing with vectors)
- Solution: Operator overloading
- See, e.g., Langtangen ch 7.3-7.5 (4th edition)
- **Overloading**: Giving an operation a (new) meaning.

### Overloading in Python

- All classes inherit from `object` methods for
    - initialization (constructor)
    - string representation (printing)
    - comparison (by `id`)
    - etc
- Operations are implemented by `__xxxxxx__()` methods
- We can *overload* these functions to define behavior for our classes
- First example: constructor `__init__()`


### Defining mathematical operations

- `+`, `-`, `*`, `/` and further mathematical operators can be defined for classes
- See http://docs.python.org/library/operator.html for a complete list
- No default definitions are inherited from `object`: only what you provide is available
- Think carefully about what definitions may make sense, e.g.,
    - string addition: concatenation
    - string times integer n: concatenate string with itself n times
    - subtraction and division not definable for string
- Methods: `__add__`, `__sub__`, `__mul__`, `__truediv__`
- `a + b` is equivalent to `a.__add__(b)`
- Below
    - `lhs`: left-hand side
    - `rhs`: right-hand side

In [1]:
import math

In [2]:
class Vector:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'

    def __add__(self, rhs):
        return Vector(self.x + rhs.x, self.y + rhs.y)

    def __sub__(self, rhs):
        return Vector(self.x - rhs.x, self.y - rhs.y)

    def __mul__(self, rhs):
        return Vector(self.x * rhs, self.y * rhs)

    def __rmul__(self, lhs):
        return self * lhs

    def __truediv__(self, rhs):
        return self * (1. / rhs)

    def norm(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

We create a few vectors and work with them. Note that `print` now falls back on the `__repr__()` method for printing the vectors, because `__str__()` is not implemented.

In [3]:
v = Vector(1, 2)
w = Vector(30, 40)

print("v       = ", v)
print("w       = ", w)
print("v + w   = ", v + w)
print("v * 5   = ", v * 5)
print("2 * v   = ", 2 * v)
print("v / 10  = ", v / 10)
print("norm(v) = ", v.norm())

v       =  Vector(1, 2)
w       =  Vector(30, 40)
v + w   =  Vector(31, 42)
v * 5   =  Vector(5, 10)
2 * v   =  Vector(2, 4)
v / 10  =  Vector(0.1, 0.2)
norm(v) =  2.23606797749979


- `__rmul__()` vs `__mul__()`:
    - `v * 5` is `v.__mul__(5)`: no problem, run `Vector.__mul__(v, 5)`
    - `2 * v` would be `2.__mul__(v)`, i.e., `int.__mul__(2, v)`
    - `int` knows nothing about vectors: error!
    - `__rmul__()`: called with swapped arguments if `__mul__()` fails
    - `2 * v` becomes `v.__rmul__(2)`, running `Vector.__rmul__(v, 2)`
    - `__rmul__()` usually implemented in terms of `__mul__()` or `*`
- `r`-versions also for other math methods

### Overriding comparisons

- `<`, `<=`, `>`, `>=`, `==`, `!=` can be overriding by defining `__lt__`, `__le__`, `__gt__`, `__ge__`, `__eq__`, `__ne__`
- `x < y` is equivalent to `x.__lt__(y)`
- Shall return `True` or `False`
- This set of six comparisons is known as *rich comparisons*

#### Default comparisons

- By default, a new class inherits comparisons from the fundamental base class `object`
- `__eq__` and `__ne__` test for *object identity*
    - same `o1 == o2` means the same as `o1 is o2`
    - this may lead to confusing results, because we expect semantic comparison from `==`
- All other comparisons return `NotImplemented` and will result in an error

In [4]:
o1 = object()
o2 = object()

o1 == o1, o1 == o2

(True, False)

In [5]:
o1 < o2

TypeError: '<' not supported between instances of 'object' and 'object'

#### Class-specific comparisons

- Override only comparisons that can be defined meaningfully!
- Equality can be defined for most types
- Only define "less than" and similar if there is one universal way of ordering instances of a class
    - Numbers are well-ordered in a mathematical sense: define `__lt__()` etc
    - Vectors can only be compared for equality
    - If instances can be ordered in different ways in different situations (by name, member number, age, ...) define the ordering rule as `key` to the sorting function
- If you define "less than", implement all other comparisons as well
    - Define them in terms of `<` and `==` to ensure consistency
    
##### Example: Vector class

- Only equality and inequality
- Try first vector class from above *without* comparisons

In [6]:
v1 = Vector(1, 2)
v2 = Vector(1, 2)
v1 == v2

False

- Result is `False` because `Vector` inherited `__eq__` from `object` and tests for `v1 is v2`
- Now create class with overridden comparisons

In [7]:
class NewVector:

    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, rhs):
        return self.x == rhs.x and self.y == rhs.y
    
    def __ne__(self, rhs):
        return not ( self == rhs )

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'

    def __add__(self, rhs):
        return Vector(self.x + rhs.x, self.y + rhs.y)

    def __sub__(self, rhs):
        return Vector(self.x - rhs.x, self.y - rhs.y)

    def __mul__(self, rhs):
        return Vector(self.x * rhs, self.y * rhs)

    def __rmul__(self, lhs):
        return self * lhs

    def __truediv__(self, rhs):
        return self * (1. / rhs)

    def norm(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

In [8]:
nv1 = NewVector(1, 2)
nv2 = NewVector(1, 2)
nv3 = NewVector(1.0, 2.0)
nv4 = NewVector(5, 8)

print(f"{nv1 == nv2 = }")
print(f"{nv1 == nv3 = }")
print(f"{nv1 == nv4 = }")

nv1 == nv2 = True
nv1 == nv3 = True
nv1 == nv4 = False


- We now compare for equality in the mathematical sense
- We do not allow relative comparisons

In [9]:
nv1 < nv4

TypeError: '<' not supported between instances of 'NewVector' and 'NewVector'

In [10]:
vecs = [NewVector(1, 2), NewVector(7, 1), NewVector(2, 8), NewVector(6, 0)]

- If we want to sort vectors, we can provide an explicit sorting criterium
- We could, e.g., sort by length

In [12]:
sorted(vecs, key=lambda v: v.norm())

[Vector(1, 2), Vector(6, 0), Vector(7, 1), Vector(2, 8)]

- Or we can sort by x-coordinate

In [13]:
sorted(vecs, key=lambda v: v.x)

[Vector(1, 2), Vector(2, 8), Vector(6, 0), Vector(7, 1)]

- By providing the sort key explicitly as argument to the sort function, anyone reading the code immediately sees which sorting criterium is used.
- This improves code robustness in all cases where there is no "fully agreed" order for objects.

#### Example: a fraction class supporting all comparisons

- We want a class expressing fractions as integer numerators and denominators to avoid round-off errors
- For mathematical fractions, order is well defined in the mathematical sense
- Therefore, it makes sense to implement comparison operators
- Note that we only need to implement `__eq__()` and `__lt__()` explicitly
- All other comparisons can be constructed from those two.

$$\frac{a}{b}<\frac{c}{d}\Leftrightarrow a d < c b \;, b, d > 0$$

In [14]:
class Fraction:
    def __init__(self, a, b):
        assert b > 0, "Denominator b > 0 required."
        self.a, self.b = a, b
    
    def __eq__(self, rhs):
        return self.a * rhs.b == rhs.a * self.b
    
    def __ne__(self, rhs):
        return not ( self == rhs )
    
    def __lt__(self, rhs):
        return self.a * rhs.b < rhs.a * self.b  # expand fractions to same denominator and compare numerators

    def __le__(self, rhs):
        return self < rhs or self == rhs
    
    def __gt__(self, rhs):
        return rhs < self

    def __ge__(self, rhs):
        return rhs <= self

In [15]:
Fraction(1, 2) == Fraction(2, 4)

True

In [16]:
f1 = Fraction(3, 4)
f2 = Fraction(2, 3)

In [17]:
print(f"{f1 == f2 = }")
print(f"{f1  < f2 = }")
print(f"{f1 >= f2 = }")

f1 == f2 = False
f1  < f2 = False
f1 >= f2 = True


- How about implementing the equality check as

    ```python
    self.a == rhs.a and self.b == rhs.b
    ```

------------------------------------------

## Names, Namespaces, and Scopes

### Why worry about names?

1. Programs execute functions to manipulate data.
1. Data and functions are stored as sequences of bits in memory.
1. We need *names* to refer to data and functions in our programs.
1. In large programs
    - the same name may be used or different purposes in different places
    - it is impossible to keep an overview over all names
    - E.g.: what do you get if you run `from xyz import *`?
1. Solution: *namespaces* and *scoping rules*

#### Namespaces (navnerom)
Namespaces help to keep names organized.

#### Scoping rules (regler for gyldighetsområder)
Scoping rules define which namespace applies in each part of a program.

### How do we bind names to objects in Python?

Operation  |  Example  | Name bound
:- | :- | -
Assignment | `x = 2`| `x`
Function definition | `def f(): pass` | `f`
Class definition | `class A: pass`  | `A`
Module import | `import math` | `math`
| | `import math as m` | `m` 
| | `from math import sin` | `sin` 

[Code on Pythontutor](https://pythontutor.com/visualize.html#code=x%20%3D%202%0A%0Adef%20f%28%29%3A%0A%20%20%20%20pass%0A%0Aclass%20A%3A%0A%20%20%20%20pass%0A%0Aimport%20math%0Aimport%20math%20as%20m%0Afrom%20math%20import%20sin&cumulative=false&heapPrimitives=true&mode=edit&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

#### Definitions are statements

Definitions are statements in Python programs. They are executed just as all other statements.

#### Presentation on topic

- See also https://nedbatchelder.com/text/names1.html

### Where are names bound?

<img src="../l08/L08_NamesBound.png" width="60%">

[Code on Pythontutor](http://www.pythontutor.com/visualize.html#code=class%20Friend%3A%0A%20%20%20%20%0A%20%20%20%20greeting%20%3D%20'Hi,%20'%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20name%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20def%20greet%28self%29%3A%0A%20%20%20%20%20%20%20%20text%20%3D%20Friend.greeting%20%2B%20self.name%0A%20%20%20%20%20%20%20%20print%28text%29%0A%20%20%20%20%20%20%20%20%0Ajoe%20%3D%20Friend%28'Joe'%29%0Ajoe.greet%28%29%0A&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

### Name-binding rules

- When a name is bound, it is registered in *exactly one namespace*
- Available namespaces
    - `__builtin__` namespace of Python interepreter
    - each module has a namespace (each imported `*.py` file)
    - each class has a namespace
    - each class *instance* has a namespace
    - each function *invocation* has a namespace
- In which namespace is a name registered?
    - In the namespace of the *innermost scope*
    - Inside a list comprehension: in the comprehension's namespace
    - Inside function definitions: in the function invocation's namespace
    - Inside class definitions: in the class' namespace
    - Otherwise, in the module's namespace
- Example of name binding in recursive function calls: 
[Code on Pythontutor](http://pythontutor.com/visualize.html#code=def%20factorial%28n%29%3A%0A%20%20%20%20if%20n%20%3C%3D%201%3A%0A%20%20%20%20%20%20%20%20res%20%3D%201%0A%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20res%20%3D%20n%20*%20factorial%28n-1%29%0A%20%20%20%20return%20res%0A%20%20%20%20%20%20%20%20%0Aprint%28factorial%283%29%29%0A&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

### Where are names looked up?

<img src="../l08/L08_NamesLookup.png" width="60%">

### Name-lookup rules

- When we use a name, Python must loop up the name and find the object it refers to
- In which namespace does Python look?
- **LEGB rule** (Mark Lutz, *Learning Python*)
    - **L—Local:** Namespace of the function invocation currently executing
    - **E—Enclosing:** Namespaces of all functions enclosing the definition of the current function (ignore for now)
    - **G—Global:** Namespace of module in which the current function *was defined*
    - **B—Builtin:** Namespace of Python builtins
- Exceptions can be forced with `global` and `nonlocal` keywords (avoid for now)

### Attribute lookup with "dot"

- Modules, classes, and instances have attributes
- Attribute names are bound inside module, class, instancance namespace
- Accessible through the dot-operator:

In [18]:
class Friend:
    
    greeting = 'Hi, '
    
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        text = Friend.greeting + self.name
        print(text)
        
joe = Friend('Joe')
print(joe.name)

Joe


In [19]:
joe.name = 'Joe Doe'
joe.greet()

Hi, Joe Doe


In [20]:
Friend.greeting = 'Hello, '
joe.greet()

Hello, Joe Doe


### Attribute lookup in instances vs classes

<img src="../l08/L08_NamesInstanceClass.png" width="45%">

### Pitfall: Duplicate attribute names

- Python uses the same namespace for methods and data attributes
- Names are looked up in the instance namespace first, then in the class namespace
- This may lead to surprises when using the same name in multiple places

In [21]:
class Friend:
    def __init__(self, name):
        self.name = name
    def greet(self):
        print('Hi,', self.name)
    def name(self):
        print('Your name is', self.name)

In [22]:
joe = Friend('Joe')
joe.greet()

Hi, Joe


In [23]:
joe.name()

TypeError: 'str' object is not callable

- What happended here?
    - Lookup starts with the instance, where we find the string stored in `self.name``
    - Then Python tries to call this string as a function because of the `()` after `joe.name`
    - Lookup stops at first match and thus never sees the method `name(self)` in this case

[Code on Pythontutor](http://www.pythontutor.com/visualize.html#code=class%20Friend%3A%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20name%0A%20%20%20%20def%20greet%28self%29%3A%0A%20%20%20%20%20%20%20%20print%28'Hi,',%20self.name%29%0A%20%20%20%20def%20name%28self%29%3A%0A%20%20%20%20%20%20%20%20print%28'Your%20name%20is',%20self.name%29%0A%20%20%20%20%20%20%20%20%0Ajoe%20%3D%20Friend%28'Joe'%29%0Ajoe.greet%28%29%0Ajoe.name%28%29&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

------

## Copying objects

### Assignment only adds new name to existing object

In [24]:
class A:
    pass
a = A()
a.x = 10
b = a
b.x = 20
print(a.x, b.x)

20 20


- `a` and `b` are two names for the same object
- [Code on PythonTutor](http://www.pythontutor.com/visualize.html#code=class%20A%3A%0A%20%20%20%20pass%0Aa%20%3D%20A%28%29%0Aa.x%20%3D%2010%0Ab%20%3D%20a%0Ab.x%20%3D%2020%0Aprint%28a.x,%20b.x%29%0A&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

### The Python `copy` module

- `copy` provides functions for copying objects
- [Code on PythonTutor](http://www.pythontutor.com/visualize.html#code=import%20copy%0Aclass%20A%3A%0A%20%20%20%20pass%0Aa%20%3D%20A%28%29%0Aa.x%20%3D%2010%0Ab%20%3D%20a%0Ab.x%20%3D%2020%0Ac%20%3D%20copy.copy%28a%29%0Ac.x%20%3D%2050%0Aprint%28a.x,%20b.x,%20c.x%29%0A&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

In [25]:
import copy
c = copy.copy(a)
c.x = 50
print(a.x, b.x, c.x)

20 20 50


#### Copying: Details

In [26]:
class S:
    pass

s = S()
s.m = 'Nice weather today.'
t = copy.copy(s)
print(hex(id(s)), hex(id(t)))
print(hex(id(s.m)), hex(id(t.m)))

0x10a0a6270 0x10a2560d0
0x10a70fe70 0x10a70fe70


- `s` and `t` have *different* `id()`: they are *different `S` instances*
- `s.m` and `t.m` have the same `id()`: they are the *same string instance*
- `copy.copy()` is a *shallow copy*: `t` is a new instance with its own namespace, but the names refer to the same objects as in `s`
- Assignment to a member re-binds the name to a new string object

In [27]:
t.m = 'The forecast for tomorrow is also nice.'
print(hex(id(s)), hex(id(t)))
print(hex(id(s.m)), hex(id(t.m)))

0x10a0a6270 0x10a2560d0
0x10a70fe70 0x10a702830


- [Explore on PythonTutor](http://www.pythontutor.com/visualize.html#code=import%20copy%0Aclass%20S%28object%29%3A%0A%20%20%20%20pass%0A%0As%20%3D%20S%28%29%0As.m%20%3D%20'Nice%20weather%20today.'%0At%20%3D%20copy.copy%28s%29%0At.m%20%3D%20'The%20forecast%20for%20tomorrow%20is%20also%20nice.'%0A&cumulative=false&curInstr=0&heapPrimitives=true&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

### Copying object with mutable members: deep copy

- Mutables: lists, dictionaries, objects of most classes
- What does shallow copy mean for objects with mutable members?

In [28]:
u = S()
u.m = [1, 2, 3]
v = copy.copy(u)
u.m.append(4)
print(u.m, v.m)
print (hex(id(u.m)), hex(id(v.m)))

[1, 2, 3, 4] [1, 2, 3, 4]
0x10a718d00 0x10a718d00


- Lists are changed in *both* `u` and `v` because their `m` refers to the same list object
- Solution: *deep copy*

In [29]:
w = copy.deepcopy(u)
print(u.m, v.m, w.m)
print (hex(id(u.m)), hex(id(v.m)), hex(id(w.m)))

[1, 2, 3, 4] [1, 2, 3, 4] [1, 2, 3, 4]
0x10a718d00 0x10a718d00 0x10a71a580


- Note that `w.m` has a different `id`

In [30]:
w.m.append(5)
print(u.m, v.m, w.m)

[1, 2, 3, 4] [1, 2, 3, 4] [1, 2, 3, 4, 5]


- Since `w.m` is a different list instance, `u.m` and `v.m` are not changed
- `copy.deepcopy()` also works for classes we write ourselves

In [32]:
class V:
    def __init__(self, x, y):
        self.x, self.y = x, y
        
class C:
    def __init__(self, ctr, r):
        self.ctr, self.r = ctr, r
        
b = C(V(0, 0), 1)
c = copy.copy(b)
d = copy.deepcopy(b)

c.ctr.x = 10
d.ctr.x = 20

print(f"{b.ctr.x = }")
print(f"{c.ctr.x = }")
print(f"{d.ctr.x = }")

b.ctr.x = 10
c.ctr.x = 10
d.ctr.x = 20


**Forgetting the difference between shallow and deep copy is a common source of errors in Python programs!**

------------

# Staying in control

## Background

- Computers can solve complex tasks fast
- Humans tend to trust in results provided by computers
- In some situations, lives depend on computers working correctly
- Requires reliable software
- Difficult to achieve: we can demonstrate the presence of bugs, proving their absence is (essentially) impossible
- Field of software engineering: *Verification* and *Validation*
- We look only at essential elements

### Elements of reliable software

- Software shall not return incorrect results
- Software shall fail in controlled ways 
- Software shall handle unforseen conditions
- Software shall be tested solidly
- **Software should fail rather than return incorrect results.**

### Techniques towards reliability

- **Assertions**
    - check that requirements are fulfilled
    - stop execution if requirement not fulfilled
    - key use cases
        - very simple "emergency stops" if we don't want to spend time on proper error handling
        - catching things that "cannot happen", but where we want to be on the safe side (in large projects, you never know ...)
- **Exceptions**
    - mechanism for signaling that something unexpected happended
    - available in most modern programming languages
    - exceptions are *raised* or *thrown* when a problem is detected
    - exceptions can be *caught* and *handled*, e.g., by issuing a useful error message
    - in some languages, e.g., Python, exceptions are also used as part of normal programming
- **Testing**
    - systematic testing of code can help us to find errors
    - a proper set of tests also helps us to avoid introducing new errors as software evolves
    - *unit tests* are tests of small parts of code, typically functions
    - *integration tests* test that the parts of a larger project work together
    - *regression tests* are added when a bug is discovered
        - the test reproduces the bug
        - when the bug is fixed, the test passes
        - we keep the test, in case we should re-introduce the bug by a later change (regress)

----------

## Assertions

- *pass* if a boolean expression is True
- *fail* if a boolean expression is False

In [33]:
assert True

In [34]:
assert False

AssertionError: 

We can use them to catch certain conditions

In [35]:
def inverse(x):
    """Returns 1 / x."""
    
    assert x != 0
    
    return 1. / x

In [36]:
inverse(10)

0.1

In [37]:
inverse(0.)

AssertionError: 

We can provide some more information to the user by adding a string after the boolean expression:

In [38]:
def inverse(x):
    """Returns 1 / x."""
    
    assert x != 0, "Inverse of 0 is not defined."
    
    return 1. / x

In [39]:
inverse(0)

AssertionError: Inverse of 0 is not defined.

We can also use this to check for conditions that are mathematically defined, but make no sense.

In [40]:
import math

def area(r):
    """Returns area of circle with radius r."""
    
    assert r >= 0, 'Circle radius must be positive.'
    
    return math.pi * r**2

In [41]:
area(1)

3.141592653589793

In [42]:
area(-1)

AssertionError: Circle radius must be positive.

- Assertions are the simplest, most brute-force way of checking conditions
- Typically used for errors that "cannot happen", but where we want to be safe

-------

## Exceptions

- Exceptions provide more fine-grained control over unexpected situations
- Python defines a number of different exception types (see [Python Documentation](https://docs.python.org/3/library/exceptions.html))
- These exception types are arranged as a class hierarchy
- The diagram shows some of the pre-defined exception types

        +-- Exception
              +-- StandardError
              |    +-- ArithmeticError
              |    |    +-- FloatingPointError
              |    |    +-- OverflowError
              |    |    +-- ZeroDivisionError
              |    +-- AssertionError
              |    +-- AttributeError
              |    +-- EnvironmentError
              |    |    +-- IOError
              |    +-- EOFError
              |    +-- ImportError
              |    +-- LookupError
              |    |    +-- IndexError
              |    |    +-- KeyError
              |    +-- NameError
              |    +-- RuntimeError
              |    |    +-- NotImplementedError
              |    +-- SyntaxError
              |    |    +-- IndentationError
              |    |         +-- TabError
              |    +-- SystemError
              |    +-- TypeError
              |    +-- ValueError
              
- We can use an exception in our `area()` function
    - We `raise` the exception: execution stops here
    - The type of exception indicates the kind of problem
    - We can provide an error message to be sent to the user

In [43]:
def area(r):
    """Returns area of circle with radius r."""
    
    if r < 0:
        raise ValueError('Circle radius must be positive.')
    
    return math.pi * r**2

In [44]:
area(10)

314.1592653589793

In [45]:
area(-5)

ValueError: Circle radius must be positive.

- Almost the same effect as an assertion

### Catching exceptions

- We can catch an exception and handle it

In [46]:
while True:
    r = float(input('Radius: '))
    if r == 0:
        break
    try:
        print('    Area:', area(r))
    except ValueError:
        print('    An error occured')

    Area: 50.26548245743669
    An error occured


- We can also extract the error message an print it

In [47]:
while True:
    r = float(input('Radius: '))
    if r == 0:
        break
    try:
        print('    Area:', area(r))
    except ValueError as err:
        print(err)

    Area: 50.26548245743669
Circle radius must be positive.


- Or even nicer for the user

In [48]:
while True:
    r = float(input('Radius: '))
    if r == 0:
        break
    try:
        print('    Area:', area(r))
    except ValueError as err:
        print(f'    ERROR: {err}\n    Please try again!')

    ERROR: Circle radius must be positive.
    Please try again!


### Separation of error detection and handling

- We *raise* an exception at the point in the code where we detect a problem.
    - Example: in the `area()` function
- We *handle* the exception where we best can regain control, e.g., by "talking" to the user
- This can be rather far away from where the error is raised—see in-class chutes example.

### Exceptions as part of normal programming (Python style)

- In certain cases, we can use exceptions to choose action
- First try something, then something else
- Consider a string containing numbers

        2 3.4 12.8 72
        
- Converting this directly into numbers would force us to make all floats

In [49]:
s = "2 3.4 12.8 72"
[float(num) for num in s.split()]

[2.0, 3.4, 12.8, 72.0]

- We can try to convert to `int` first and only convert to `float` if that fails

In [50]:
def float_or_int(s):
    try:
        return int(s)
    except ValueError:
        return float(s)

[float_or_int(num) for num in s.split()] 

[2, 3.4, 12.8, 72]

-------

## Testing Python code

- Systematic testing is essential part of quality control
- Do not trust code that comes without tests!
- But: passings tests are no guarantee that everything is correct
    - Tests might not cover all code
    - Tests may not cover all possible situations
    - Tests may pass for the wrong reasons
- Different levels of testing
    - Unit tests: test "units", i.e., functions and methods
    - Integration tests: test larger parts, e.g., modules or packages
    - Acceptance tests: tests by client required for accepting delivery, test the entire system against client use-cases
    - Regression tests: test detecting bugs in earlier versions, kept to avoid falling back to old mistakes

### Agile software development

- [Agile software development](https://en.wikipedia.org/wiki/Agile_software_development) is a modern (2001-) set of software development methods
- Focus on quick delivery, frequent updates, and flexibility, while maintaining quality
- Strong focus on testing
- [Test-driven development](https://en.wikipedia.org/wiki/Test-driven_development) is part of Agile
    - Write tests first, otherwise you'll never write them
    - Writing tests for working code often results in weak tests
    - Therefore
        1. Write test
        1. Write code only when a test fails
        1. Write code until tests passes (and no more)
        1. Review and refactor code without breaking tests
        1. Go back to 1
    - Important: Always run all tests after code changes
- Advantages
    - We can be much more confident in our code
    - We can make changes, big and small, and immediately check that the code still works correctly

### Tools for testing

- Systems for automatically running tests on changes or commits
    - often known as [continuous integration](https://en.wikipedia.org/wiki/Continuous_integration)
    tools
    - "watch" VCS repository and run tests on each commit or push
    - notify developers in case of trouble
    - can be combined with code-review platforms
    - [GitLab Pipelines](https://about.gitlab.com/blog/2019/07/12/guide-to-ci-cd-pipelines/), [Jenkins](http://jenkins-ci.org), [GitHub Actions](https://github.com/features/actions)

### Writing tests

There are several tools (frameworks) for writing and running tests in Python

- [unittest](https://docs.python.org/3/library/unittest.html)
    - advantage: part of standard Python
    - disadvantage: no automated test discovery
- [nosetest](https://nose.readthedocs.org/en/latest/testing.html)
    - advantage: automated test discovery
    - disadvantage: no longer actively maintained (nor is nosetest2)
- [pytest](https://docs.pytest.org/en/latest/index.html)
    - automated test discovery
    - easy coding style
    - actively developed
    - powerful advanced features
- We will use pytest 

### pytest basics

- pytest finds tests automatically
- Any file named `test_*.py` or `*_test.py` will be considered a collection of tests
- In such files
    - any function called `test_*`
    - any method called `test_*` in a class called `Test*`
  will be run as tests
- See [pytest documentation](https://docs.pytest.org/en/latest/goodpractices.html) for more on test discovery
- A test *passes* if it does not throw an exception
- In pytest, all checks are implemented as `assert`
- Module `pynest` provides useful tools

### Tests and randomness

Tests including random numbers introduce particular problems; we will not discuss these here.

### A Pytest example

#### Preparations

- You first need to `pip install pytest pytest-coverage`

#### Running a first test

- Test examples are in directory `tests`
- We will start with `test_example_buggy.py` 
- Run this set of tests in the terminal with
    ```
    cd lectures/l09
    pytest tests/test_example_buggy.py
    ```
- On my computer, I got the following results

##### Results from `test_example.py`

```
(.venv) l09> pytest tests/test_example_buggy.py
====================== test session starts ======================================
platform darwin -- Python 3.13.9, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/plesser/Courses/INF201/H2025/inf201-course-materials/lectures/l09
plugins: cov-7.0.0
collected 2 items                                                                                                

tests/test_example_buggy.py .F                                             [100%]

=========================== FAILURES ============================================
________________________ test_square_2 __________________________________________

    def test_square_2():
>       assert square(2) == 4
E       assert 8 == 4
E        +  where 8 = square(2)

tests/test_example_buggy.py:22: AssertionError
===================== short test summary info ====================================
FAILED tests/test_example_buggy.py::test_square_2 - assert 8 == 4
==================== 1 failed, 1 passed in 0.04s =================================
```



- We see that one test failed, one passed
- Corrected code is in `test_example_fixed.py`
- Both tests pass for this code

### Testing for exceptions

- In some cases, we want to test that a function raises an exception
- In Pytest, with can do this with the help of a *context*
- `test_division.py` gives an example of how to test that a `ValueError` is raised for an unacceptable argument value

### Parameterizing tests

- Often, we want to perform the same test for different argument values
- Test parameterization makes this easy in PyTest
- See also https://docs.pytest.org/en/stable/example/parametrize.html

#### Example: Testing the `square()` function

##### Explicit tests for each argument

```python
def test_square_1():
    assert square(1) == 1

def test_square_2():
    assert square(2) == 4
```

##### Parameterized tests

```python
@pytest.mark.parametrize("x, expected",
                         [[1, 1],
                          [2, 4]
                          ])
def test_square(x, expected):
    assert square(x) == expected
```

- `@pytest.mark.parametrize()` is a *decorator*
- The first decorator argument is a string with comma-separated variable names
- The second decorator argument is a list
    - Each list element represents one test case
    - Each list element must be a list (or tuple) with as many elements as variable names in the string
    - The variables need to be passed to the test function
- Pytest runs each case as a separate test
- See `test_parametrize.py` for an example
- Question: Which other cases could we add to the tests for `square()`?

### Test discovery

- So far, we have run individual tests from the Terminal
- Usually, we want to run a whole set of tests for a project
- PyTest provides automatic *test discovery*
    - If we run it with a directory, it will find all files starting with `test*`
    - Inside those files, it will look for all functions starting with `test*` and run them as tests
    - We can also create classes called `Test*`, but we won't look into that here
    - If we run pytest without a directory name, it will search the current directory
- **Note**: If you run `pytest` on the entire `inf201-course-materials` folder or the entire `lectures` folder, you will get and "error during collection" and no tests will run. This is because `lectures/l02` contains some `testfile_*.py` files, which PyTest searches for tests, and at least one of those is (for demonstration purposes in Lecture 2) not correctly UTF-8 encoded. The solution in this case is not to run `pytest` on the entire `lectures` folder.

### Test coverage

- It is important to know how much code is actually tested
- This is called *test coverage*
- Complete coverage can be difficult to achieve in complex programs
- Even complete coverage does not guarantee absence of errors
- To measure coverage, run `pytest --cov tests`
- Here the result for the example tests:
    ``` 
    Name                          Stmts   Miss  Cover
    -------------------------------------------------
    tests/test_division.py            8      1    88%
    tests/test_example_buggy.py       6      0   100%
    tests/test_example_fixed.py       6      0   100%
    tests/test_parametrize.py         6      0   100%
    -------------------------------------------------
    TOTAL                            26      1    96%
    ```
- Note: Coverage measurement only works for directories, not for individual test files

### Pytest in Visual Studio Code

- We can run tests directly from Pytest
- Requires configuration
- See video "INF201_H25_VSCodeTesting"
- See https://code.visualstudio.com/docs/python/testing


### More on testing

- The video "INF200_H22_PytestExample" provides an extended example of writing tests for the Fraction class.
- The corresponding code is available under `l09/fraction`.
- Note that in the video, I use a different IDE (PyCharm instead of VSCode). I hope this does not distract from the code development. Also, the code available here is the final version at the end of the video.